### Huấn luyện mô hình với SVM (26/03/2025)
Thay đổi chính:
  1. Sử dụng tệp CSV.
  2. Sử dụng Kernel RBF để xử lý dữ liệu không tuyến tính cho bài toán trái cây.
  3. Sử dụng StandardScaler để chuẩn hoá dữ liệu về cùng thang đo.
  4. Áp dụng PCA để giảm chiều dữ liệu từ 1284 -> nhỏ hơn (VD: 50) => Giảm độ phức tạp tính toán và tăng tốc độ huấn luyện.
  
  PCA giữ lại phần lớn thông tin quan trọng (phương sai), nên không làm mất quá nhiều dữ liệu.

Kết quả:
  - Mô hình đã được huấn luyện nhanh hơn.
  - Tỉ lệ chính xác chỉ 50%.

In [25]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import pandas as pd
import joblib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Đọc dữ liệu từ tệp CSV

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SVM Training/features.csv')
X = data.drop("Label", axis=1)
y = data["Label"]

Chuẩn hoá dữ liệu

In [8]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Áp dụng PCA

In [13]:
pca = PCA(n_components=4)  # Giảm xuống 50 chiều
X_pca = pca.fit_transform(X_scaled)
print(f"Phần trăm phương sai được giữ lại: {sum(pca.explained_variance_ratio_):.4f}")

Phần trăm phương sai được giữ lại: 1.0000


Chia tập dữ liệu Train/Test

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Huấn luyện mô hình SVM

In [20]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

SVC()

Tìm tham số tối ưu

In [23]:
# Tìm tham số tối ưu
param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.01, 0.1]}
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Tham số tối ưu:", grid_search.best_params_)
print("Độ chính xác tốt nhất:", grid_search.best_score_)

Tham số tối ưu: {'C': 1, 'gamma': 0.01}
Độ chính xác tốt nhất: 0.6717229376957884


Đánh giá mô hình

In [24]:
accuracy = grid_search.score(X_test, y_test)
print('Độ chính xác:', accuracy)

Độ chính xác: 0.6933962264150944


Tải mô hình với định dạng .pkl (tệp pickle)
> SVM không phải là mô hình deep learning như MobileNetV2, nên nó không được lưu trực tiếp dưới dạng .h5 hoặc .tflite (định dạng thường dùng cho các mô hình TensorFlow/Keras). Thay vào đó, SVM (từ sklearn) thường được lưu dưới dạng tệp .pkl (pickle) hoặc các định dạng tương tự.

In [28]:
import joblib

# Lưu mô hình SVM đã huấn luyện
svm_model_path = '/content/drive/MyDrive/Colab Notebooks/SVM_FruitsClassifi_26032025.pkl'
joblib.dump(grid_search, svm_model_path)
print(f"Đã lưu mô hình SVM tại: {svm_model_path}")

Đã lưu mô hình SVM tại: /content/drive/MyDrive/Colab Notebooks/SVM_FruitsClassifi_26032025.pkl


Sử dụng mô hình

In [37]:
# Import lại nếu cần
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [29]:
# Tải mô hình SVM (Tải lại nếu cần)
svm_model_path = '/content/drive/MyDrive/Colab Notebooks/SVM_FruitsClassifi_26032025.pkl'
svm_model = joblib.load(svm_model_path)

In [31]:
# Tải dữ liệu kiểm thử (tệp features.csv chứa đặc trưng đã trích xuất)
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SVM Training/features.csv')
X_test = test_data.drop(['Label'], axis=1)  # Loại bỏ cột Label
y_test = test_data['Label']

In [35]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

In [38]:
# Dự đoán trên tập kiểm thử
predictions = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
print(f"Độ chính xác trên tập kiểm thử: {accuracy:.4f}")

Độ chính xác trên tập kiểm thử: 0.3239


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [42]:
# In kết quả dự đoán cho một số mẫu
class_labels = ['Apple Ripe', 'Banana Ripe', 'Orange Ripe']

# Kiểm tra xem cột 'Filename' có tồn tại trong test_data không
if 'Filename' in test_data.columns:
    for i in range(min(5, len(predictions))):
        print(f"Ảnh {test_data['Filename'].iloc[i]}: Thực tế: {class_labels[y_test.iloc[i]]}, Dự đoán: {class_labels[predictions[i]]}")
else:
    print("Cột 'Filename' không tồn tại trong DataFrame. In kết quả dự đoán theo chỉ số:")
    for i in range(min(5, len(predictions))):
        # Convert labels to numerical indices before accessing class_labels
        actual_label_index = class_labels.index(y_test.iloc[i])
        predicted_label_index = class_labels.index(predictions[i])
        print(f"Mẫu {i}: Thực tế: {class_labels[actual_label_index]}, Dự đoán: {class_labels[predicted_label_index]}")

Cột 'Filename' không tồn tại trong DataFrame. In kết quả dự đoán theo chỉ số:
Mẫu 0: Thực tế: Banana Ripe, Dự đoán: Banana Ripe
Mẫu 1: Thực tế: Banana Ripe, Dự đoán: Banana Ripe
Mẫu 2: Thực tế: Banana Ripe, Dự đoán: Banana Ripe
Mẫu 3: Thực tế: Banana Ripe, Dự đoán: Banana Ripe
Mẫu 4: Thực tế: Banana Ripe, Dự đoán: Banana Ripe
